In [ ]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
# from quantuminspire.projectq.backend_qx import QIBackend

import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

In [ ]:
num_pts = 100

coords=[]

coords.append((0,0))
for i in range(int(num_pts/2)-1):
    coords.append((np.pi/(num_pts/2)*(i+1),0))
    coords.append((np.pi/(num_pts/2)*(i+1),-np.pi))
coords.append((np.pi,0))

x=[np.cos(point[1])*np.sin(point[0]) for point in coords]
y=[np.sin(point[1])*np.sin(point[0]) for point in coords]
z=[np.cos(point[0]) for point in coords]

#x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()

In [ ]:
def parameterized_QACM(loc):
    θ,ϕ=loc
    if ϕ == 0:
        qasm = '''
        version 1.0

        qubits 5

        # initialize the state
        Ry q[2], {0}

        Ry q[1], 0.785398163
        Ry q[3], 0.785398163
        CNOT q[2],q[1]
        CNOT q[2],q[3]
        CNOT q[1], q[2]
        CNOT q[3], q[2]

        #Rotate back and measure
        Ry q[1], {3}
        Ry q[3], {3}
        Measure_z q[1]
        Measure_z q[3]
        '''
    else: # else phi=pi
        qasm = '''
        version 1.0

        qubits 5

        # initialize the state
        Ry q[2], {0}
        Z q[2] #rotation of pi around the Z axis

        Ry q[1], 0.785398163
        Ry q[3], 0.785398163
        CNOT q[2],q[1]
        CNOT q[2],q[3]
        CNOT q[1], q[2]
        CNOT q[3], q[2]

        #Rotate back and measure
        Z q[1]
        Ry q[1], {3}
        Z q[3]
        Ry q[3], {3}
        Measure_z q[1]
        Measure_z q[3]
        '''
    return qasm.format(θ,ϕ,-ϕ,-θ)

In [ ]:
def beta1_circuit(qubit):
    qasm = '''
    version 1.0

    qubits 5

    Measure_z q[{0}]
    '''
    return qasm.format(qubit)

In [ ]:
def beta2_circuit(qubit):
    qasm = '''
    version 1.0

    qubits 5
    
    X q[{0}]
    Measure_z q[{0}]
    '''
    return qasm.format(qubit)

In [ ]:
def readout_calibration(qubit, backend_type, N_shots):
    result1 = qi.execute_qasm(beta1_circuit(qubit), backend_type=backend_type, number_of_shots=N_shots)
    result2 = qi.execute_qasm(beta2_circuit(qubit), backend_type=backend_type, number_of_shots=N_shots)
    hist1=result1.get('histogram', {})
    p0_1=hist1["0"]
    p1_1=1-p0_1
    hist2=result2.get('histogram', {})
    p0_2=hist2["0"]
    p1_2=1-p0_2
    
    mA=p0_1-p1_1
    mB=p0_2-p1_2
    
    beta0=0.5*(mA+mB)
    beta1=0.5*(mA-mB)
    print(beta0)
    print(beta1)
    return beta0, beta1

In [ ]:
N_shots=16384
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_starmon.csv",target_points)

In [ ]:
# iterate over the sphere, run task
backend_type = qi.get_backend_type_by_name('Starmon-5')

copied_data=[]
copied_data_corr=[]

index=0
recover_index=91
qubit0=1
qubit1=3
beta0_qubit0=0.0301513671875
beta1_qubit0=0.93896484375
beta0_qubit1=0.02264404296875
beta1_qubit1=0.94500732421875
for points in target_points:
    if index < recover_index:
        index=index+1
        print(str(index)+"/"+str(num_pts))
        print("Already calculated")
    else:
        if index % 100 == 0:
            print("=============================================")
            print("READOUT CALIBRATION!")
            print("=============================================")
            beta0_qubit0, beta1_qubit0 = readout_calibration(qubit0, backend_type, N_shots)
            print("Beta0, qubit0: ", beta0_qubit0)
            print("Beta1, qubit0: ", beta1_qubit0)
            beta0_qubit1, beta1_qubit1 = readout_calibration(qubit1, backend_type, N_shots)
            print("Beta0, qubit1: ", beta0_qubit1)
            print("Beta1, qubit1: ", beta1_qubit1)
            with open('readout_parameters.txt', 'a') as file:
                file.write(str(index)+"\t"+str(beta0_qubit0)+"\t"+str(beta1_qubit0)+"\t"+
                           str(beta0_qubit1)+"\t"+str(beta1_qubit1)+"\n")


        qasm=parameterized_QACM(points)
        result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
        hist=result.get('histogram', {})

        #test both the copies
        marg_prob0_qubit0=hist["0"]+hist["8"] #calculate the marginal distribution, p=p(00000)+p(01000)
        marg_prob0_qubit1=hist["0"]+hist["2"] #calculate the marginal distribution, p=p(00000)+p(00010)
        marg_prob1_qubit0=1-marg_prob0_qubit0
        marg_prob1_qubit1=1-marg_prob0_qubit1

        copied_data.append([marg_prob0_qubit0,marg_prob1_qubit0, marg_prob0_qubit1, marg_prob1_qubit1])
        index=index+1
        print(str(index)+"/"+str(num_pts))
        print("Execution time: ", result.get('execution_time_in_seconds',{}))
        print(points)
        print("Qubit 0: ", [marg_prob0_qubit0,marg_prob1_qubit0])
        print("Qubit 1: ", [marg_prob0_qubit1,marg_prob1_qubit1])
        with open('results_starmon.txt', 'a') as file:
            file.write(str(points[0])+"\t"+str(points[1])+"\t"+str(marg_prob0_qubit0)+"\t"+str(marg_prob1_qubit0)+"\t"+
                       str(marg_prob0_qubit1)+"\t"+str(marg_prob1_qubit1)+"\n")

        #readout calibration, correct measurement
        marg_prob0_qubit0_corr= (beta1_qubit0-beta0_qubit0 + marg_prob0_qubit0 -  marg_prob1_qubit0)/(2*beta1_qubit0)
        marg_prob1_qubit0_corr= 1-marg_prob0_qubit0_corr
        marg_prob0_qubit1_corr= (beta1_qubit1-beta0_qubit1 + marg_prob0_qubit1 -  marg_prob1_qubit1)/(2*beta1_qubit1)
        marg_prob1_qubit1_corr= 1-marg_prob0_qubit1_corr
        print("Qubit 0 (corrected): ", [marg_prob0_qubit0_corr,marg_prob1_qubit0_corr])
        print("Qubit 1 (corrected): ", [marg_prob0_qubit1_corr,marg_prob1_qubit1_corr])
        with open('results_starmon_corrected.txt', 'a') as file:
            file.write(str(points[0])+"\t"+str(points[1])+"\t"+str(marg_prob0_qubit0_corr)+"\t"+str(marg_prob1_qubit0_corr)+"\t"+
                       str(marg_prob0_qubit1_corr)+"\t"+str(marg_prob1_qubit1_corr)+"\n")
        copied_data_corr.append([marg_prob0_qubit0_corr,marg_prob1_qubit0_corr, marg_prob0_qubit1_corr, marg_prob1_qubit1_corr])




copied_data=np.array(copied_data)
copied_data_corr=np.array(copied_data_corr)

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,copied_data)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])
data_sheet_corr=pd.DataFrame(data=np.hstack((target_points,copied_data_corr)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [ ]:
data_sheet.head()
data_sheet_corr.head()

In [ ]:
# save to excel, optional
data_sheet.to_excel("data_sheet_starmon.xlsx")
data_sheet_corr.to_excel("data_sheet_starmon_corr.xlsx")

In [ ]:
data_sheet=pd.read_excel("data_sheet_starmon.xlsx")
data_sheet_corr=pd.read_excel("data_sheet_starmon_corr.xlsx")

In [ ]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_1/(5/6), template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_1))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_1))

In [ ]:
#FIDELITY OF SECOND COPY
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0/(5/6), template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

In [ ]:
#CHECK SYMMETRY, I.E. PLOT fidelity of qubit 0/fidelity of qubit 2

df = px.data.iris()
ratio=data_sheet.prob_0_qubit_1/data_sheet.prob_0_qubit_0
fig = px.scatter_3d(df, x, y, z, color=ratio, template="simple_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(ratio))
print("Standard deviation: ", np.std(ratio))